In [13]:
from stravalib.client import Client
client=Client()

CLIENT_ID = '57755'
CLIENT_SECRET = '43f7beec9947a8589078e054a9c4cce465d1c870'
REDIRECT_URL = 'http://localhost:8000/authorized'

In [15]:
url = client.authorization_url(
    client_id=CLIENT_ID, redirect_uri='http://127.0.0.1:5000/authorization', 
    scope=['read_all','profile:read_all','activity:read_all']
)
url

'https://www.strava.com/oauth/authorize?client_id=57755&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fauthorization&approval_prompt=auto&response_type=code&scope=read_all%2Cprofile%3Aread_all%2Cactivity%3Aread_all'

In [16]:
import pickle

CODE = '627ecb623dc4423190462c8acf539e53b5488b9e'

access_token = client.exchange_code_for_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, code=CODE)
with open('../access_token.pickle', 'wb') as f:
    pickle.dump(access_token, f)

In [17]:
with open('../access_token.pickle', 'rb') as f:
    access_token = pickle.load(f)
    
print('Latest access token read from file:')
access_token

Latest access token read from file:


{'access_token': '65efb19abfd2c7baa9782447fde3d6b17cceaf88',
 'refresh_token': '742610f225e2f1bac22d721795390e7ef9ed1df6',
 'expires_at': 1607388883}

In [18]:
if time.time() > access_token['expires_at']:
    print('Token has expired, will refresh')
    refresh_response = client.refresh_access_token(client_id=MY_STRAVA_CLIENT_ID, client_secret=MY_STRAVA_CLIENT_SECRET, refresh_token=access_token['refresh_token'])
    access_token = refresh_response
    with open('../access_token.pickle', 'wb') as f:
        pickle.dump(refresh_response, f)
    print('Refreshed token saved to file')
    client.access_token = refresh_response['access_token']
    client.refresh_token = refresh_response['refresh_token']
    client.token_expires_at = refresh_response['expires_at']
        
else:
    print('Token still valid, expires at {}'
          .format(time.strftime("%a, %d %b %Y %H:%M:%S %Z", time.localtime(access_token['expires_at']))))
    client.access_token = access_token['access_token']
    client.refresh_token = access_token['refresh_token']
    client.token_expires_at = access_token['expires_at']

Token still valid, expires at Mon, 07 Dec 2020 18:54:43 Central Standard Time


In [19]:
athlete = client.get_athlete()
print("Athlete's name is {} {}, based in {}, {}"
      .format(athlete.firstname, athlete.lastname, athlete.city, athlete.country))

Athlete's name is Kevin Johnson, based in None, None


In [20]:
activities = client.get_activities(limit=1000)
list(activities)[0:10]

[<Activity id=4442408424 name='Afternoon Run' resource_state=2>,
 <Activity id=4442408475 name='Afternoon Activity' resource_state=2>,
 <Activity id=4442408418 name='Lunch Activity' resource_state=2>,
 <Activity id=4442408391 name='Afternoon Run' resource_state=2>,
 <Activity id=4442408413 name='Afternoon Activity' resource_state=2>,
 <Activity id=4442408412 name='Afternoon Activity' resource_state=2>,
 <Activity id=4442408376 name='Afternoon Run' resource_state=2>,
 <Activity id=4442408568 name='Afternoon Run' resource_state=2>,
 <Activity id=4442408524 name='Night Run' resource_state=2>,
 <Activity id=4442408570 name='Afternoon Run' resource_state=2>]

In [21]:
my_cols =['name',
          'start_date_local',
          'type',
          'distance',
          'moving_time',
          'elapsed_time',
          'total_elevation_gain',
          'elev_high',
          'elev_low',
          'average_speed',
          'max_speed',
          'average_heartrate',
          'max_heartrate',
          'start_latitude',
          'start_longitude']

In [22]:
data = []
for activity in activities:
    my_dict = activity.to_dict()
    data.append([activity.id]+[my_dict.get(x) for x in my_cols])
    
# Add id to the beginning of the columns, used when selecting a specific activity
my_cols.insert(0,'id')

In [25]:
df = pd.DataFrame(data, columns=my_cols)
df.head()

,id,name,start_date_local,type,distance,moving_time,elapsed_time,total_elevation_gain,elev_high,elev_low,average_speed,max_speed,average_heartrate,max_heartrate,start_latitude,start_longitude
0,4442408424,Afternoon Run,2020-11-29T15:06:26,Run,8013.5,1:00:02,1:00:02,0.0,NaN,NaN,2.225,3.1,163.0,184,NaN,NaN
1,4442408475,Afternoon Activity,2020-11-29T13:58:18,Workout,0.0,0:56:31,0:56:31,0.0,NaN,NaN,0.000,0.0,114.3,136,NaN,NaN
2,4442408418,Lunch Activity,2020-11-26T11:15:27,Workout,2681.6,0:37:01,0:46:02,0.0,196.9,195.0,1.207,2.5,105.5,116,41.820520,-87.876276
3,4442408391,Afternoon Run,2020-11-25T14:55:09,Run,5674.4,0:33:03,0:33:03,0.0,NaN,NaN,2.862,3.2,155.6,169,NaN,NaN
4,4442408413,Afternoon Activity,2020-11-25T13:24:28,Workout,338.2,0:07:43,0:24:30,0.0,180.0,179.9,0.730,3.6,114.2,150,41.949457,-87.646189
